In [1]:
import torch
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output
from bindsnet.models import DiehlAndCook2015
from bindsnet.network.monitors import Monitor
from bindsnet.encoding import poisson
import time as T
import os
plt.rcParams["figure.figsize"] = (20, 20)
start_time = str(int(T.time()))

In [2]:
time = 500
plot = False
network = DiehlAndCook2015(5000, dt=1.0, norm=48.95, inh=3)
exc_monitor = Monitor(network.layers['Ae'], ['v', 's'], time=time)
network.add_monitor(exc_monitor, name='exc')

In [3]:
def log(track_n, msg):
    lfname = f'logs/{start_time}_track{track_n+1:02d}_numenta.csv'
    with open(lfname, 'a') as f:
        f.write(msg)

In [4]:
tracks = [(255*pickle.load(open('encoding/' + fn, 'rb')))
          for fn in sorted([x for x in os.listdir('encoding/')
                            if '.p' in x])]

In [ ]:
for track_n in range(len(tracks)):
    log(track_n, 'Iteration,Neuron,Spikes\n')
    track = tracks[track_n]
    for i in range(4, len(track)):
        orig = torch.cat((track[i-4], track[i-3], track[i-2], track[i-1], track[i]))
        pt = poisson(orig, time)
        
        inpts = {'X': pt}
        network.run(inpts=inpts, time=time)
        spikes = exc_monitor.get('s')
        voltage = exc_monitor.get('v')
        network.reset_()
        
        for neuron, value in enumerate(torch.sum(spikes, dim=1)):
            log(track_n, f'{i-4},{neuron},{int(value)}\n')
        
        if plot:
            fig = plt.figure(figsize=(20, 20))
            plt.subplot(2, 2, 1)
            plt.imshow(spikes, cmap='binary')
            plt.subplot(2, 2, 2)
            plt.imshow(orig.view(125, 40), cmap='gist_gray')
            plt.show()
            
        clear_output(wait=True)

In [ ]:
network.save('trained.net')